# Imports

In [1]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/Colab Notebooks"

 cnn    data_preparer.py   data_vectorizer.py  'H&M.ipynb'    train.ipynb
 data   data_reader.py	   Demo.ipynb	        __pycache__


In [3]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks")
import data_preparer
from pathlib import Path
from keras.utils import to_categorical
import torch
import tensorflow as tf
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.callbacks import LearningRateScheduler

# Setup Training

In [4]:
# Setup traing data
vectors, letters = data_preparer.get_randomized_vectors(100)
length = data_preparer.get_max_frames()

/content/drive/My Drive/Colab Notebooks/data/sub001


In [5]:
items = set(letters)

items = {
    item: idx
    for idx, item in enumerate(items)
}

for idx, item in enumerate(letters):
    letters[idx] = items[item]

letters = np.array(letters)
letters = keras.utils.to_categorical(letters)

In [6]:
#vecotrs need to be setup differently because LSTM only taskes 3 dimesions ans cnn needs 4 dimensions
# vectors = data_preparer.make_numpy(vectors, length) # for LSTM model
vectors = data_preparer.make_numpy_cnn(vectors, length) #for CNN model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(vectors, letters, test_size=0.2, random_state=13)

# Models

### **1. LSTM**

In [ ]:
# # Mini Model
# lstm = keras.Sequential()
# lstm.add(keras.layers.LSTM(1))
# lstm.add(keras.layers.Dense(3, activation='sigmoid'))

In [ ]:
# # Small Model
# lstm = keras.Sequential()
# lstm.add(keras.layers.LSTM(20))
# lstm.add(keras.layers.Dense(10, activation='sigmoid'))
# lstm.add(keras.layers.Dense(3, activation='sigmoid'))

In [ ]:
# # Big Model
# lstm = keras.Sequential()
# lstm.add(keras.layers.LSTM(1860))
# lstm.add(keras.layers.Dense(100, activation='sigmoid'))
# lstm.add(keras.layers.Dense(10, activation='sigmoid'))
# lstm.add(keras.layers.Dense(3, activation='sigmoid'))

In [ ]:
# model.compile(loss='mean_squared_error',
#               optimizer='Adamax',
#               metrics=['CategoricalAccuracy'])

In [ ]:
#model = keras.models.load_model(r"models/small")

### **2. 3D-CNN**

In [8]:
#3D CNN model
cnn = Sequential()
cnn.add(Conv3D(60, kernel_size=(3, 3, 3), activation='relu', input_shape=(7, 20, length, 1)))
cnn.add(BatchNormalization())
cnn.add(MaxPooling3D(pool_size=(2, 2, 2)))

cnn.add(Conv3D(30, kernel_size=(3, 3, 3), padding='same',activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling3D(pool_size=(2, 2, 2)))

cnn.add(Conv3D(20, kernel_size=(3, 3, 3), padding='same',activation='relu'))
cnn.add(BatchNormalization())


cnn.add(Flatten())
cnn.add(Dense(10, activation='sigmoid'))
cnn.add(Dense(3, activation='sigmoid'))
# Compile the model
cnn.compile(loss='mean_squared_error',
              optimizer='Adamax',
              metrics=['CategoricalAccuracy'])

In [10]:
#model.save('/content/drive/My Drive/Colab Notebooks/cnn/')

### **3. 3D-CNN-LSTM**

In [12]:
# Build the model
cnn_lstm = Sequential()

# Add a 3D-CNN layer
cnn_lstm.add(Conv3D(60, kernel_size=(3, 3, 3), activation='relu', input_shape=(7, 20, length, 1)))
cnn_lstm.add(BatchNormalization())
cnn_lstm.add(MaxPooling3D(pool_size=(2, 2, 2)))
cnn_lstm.add(Dropout(0.25))

cnn_lstm.add(Conv3D(30, kernel_size=(3, 3, 3), padding='same',activation='relu'))
cnn_lstm.add(BatchNormalization())
cnn_lstm.add(MaxPooling3D(pool_size=(2, 2, 2)))
cnn_lstm.add(Dropout(0.25))

cnn_lstm.add(Conv3D(20, kernel_size=(3, 3, 3), padding='same',activation='relu'))
cnn_lstm.add(BatchNormalization())
cnn_lstm.add(Dropout(0.25))


# Flatten the output from 3D-CNN to 2D for LSTM input
cnn_lstm.add(TimeDistributed(Flatten()))

# Add an LSTM layer
cnn_lstm.add(LSTM(20))
cnn_lstm.add(Dense(10, activation='sigmoid'))
cnn_lstm.add(Dense(3, activation='sigmoid'))

# Compile the model
cnn_lstm.compile(loss='mean_squared_error', optimizer='Adamax', metrics=['CategoricalAccuracy'])

# Training

In [ ]:
# Train the model lstm


lstm.fit(X_train, y_train, epochs=10, verbose=1,batch_size=4)

In [9]:
# Train the model cnn -> takes very long on google colab


cnn.fit(X_train, y_train, epochs=10, verbose=1,batch_size=4)

Epoch 1/10
3600/3600 [==============================] - 1159s 321ms/step - loss: 0.0999 - categorical_accuracy: 0.8163
Epoch 2/10
3600/3600 [==============================] - 1151s 320ms/step - loss: 0.0501 - categorical_accuracy: 0.9076
Epoch 3/10
3600/3600 [==============================] - 1148s 319ms/step - loss: 0.0379 - categorical_accuracy: 0.9281
Epoch 4/10
3600/3600 [==============================] - 1149s 319ms/step - loss: 0.0303 - categorical_accuracy: 0.9428
Epoch 5/10
3600/3600 [==============================] - 1147s 319ms/step - loss: 0.0242 - categorical_accuracy: 0.9550
Epoch 6/10
3600/3600 [==============================] - 1145s 318ms/step - loss: 0.0226 - categorical_accuracy: 0.9567
Epoch 7/10
3600/3600 [==============================] - 1145s 318ms/step - loss: 0.0181 - categorical_accuracy: 0.9657
Epoch 8/10
3600/3600 [==============================] - 1144s 318ms/step - loss: 0.0164 - categorical_accuracy: 0.9694
Epoch 9/10
3600/3600 [==========================

In [13]:
# Train the model cnn and lstm-> takes very long on google colab


cnn_lstm.fit(X_train, y_train, epochs=10, verbose=1,batch_size=4)

Epoch 1/10
3600/3600 [==============================] - 545s 150ms/step - loss: 0.1580 - categorical_accuracy: 0.7033
Epoch 2/10
3600/3600 [==============================] - 513s 142ms/step - loss: 0.0753 - categorical_accuracy: 0.8600
Epoch 3/10
3600/3600 [==============================] - 511s 142ms/step - loss: 0.0576 - categorical_accuracy: 0.8877
Epoch 4/10
3600/3600 [==============================] - 513s 143ms/step - loss: 0.0481 - categorical_accuracy: 0.9067
Epoch 5/10
3600/3600 [==============================] - 513s 142ms/step - loss: 0.0415 - categorical_accuracy: 0.9184
Epoch 6/10
3600/3600 [==============================] - 510s 142ms/step - loss: 0.0366 - categorical_accuracy: 0.9279
Epoch 7/10
3600/3600 [==============================] - 511s 142ms/step - loss: 0.0329 - categorical_accuracy: 0.9358
Epoch 8/10
3600/3600 [==============================] - 492s 137ms/step - loss: 0.0288 - categorical_accuracy: 0.9447
Epoch 9/10
3600/3600 [==============================] - 

# Evaluation

In [ ]:
# Evaluate the model on the test set
score = lstm.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [10]:
# Evaluate the model on the test set
score = cnn.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

113/113 [==============================] - 51s 440ms/step - loss: 0.0181 - categorical_accuracy: 0.9636
Test loss: 0.01812843605875969
Test accuracy: 0.9636111259460449


In [ ]:
# Evaluate the model on the test set
score = lstm_cnn.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])